<a href="https://colab.research.google.com/github/ghulammuttaqashah/AI-Resume-Analyzer/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Necessary Libraries**

In [ ]:
!pip install pandas scikit-learn nltk
!python -m nltk.downloader stopwords

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Dataset Loading**

In [ ]:
import pandas as pd

# Define paths
splits = {'train': 'train.csv', 'test': 'test.csv'}

# Load the CSV files into DataFrames
df_train = pd.read_csv("hf://datasets/cnamuangtoun/resume-job-description-fit/" + splits["train"])
df_test = pd.read_csv("hf://datasets/cnamuangtoun/resume-job-description-fit/" + splits["test"])


# Combine the DataFrames
df_combined = pd.concat([df_train, df_test], ignore_index=True)

# Save combined DataFrame to local CSV
df_combined.to_csv("resume_job_fit_combined.csv", index=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b4e15735-ed93-4545-a993-fc97a9fd13c6)')' thrown while requesting GET https://huggingface.co/datasets/cnamuangtoun/resume-job-description-fit/resolve/main/train.csv
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: fb7eb4b6-a63a-4135-96ae-ee8b26e8753d)')'

In [ ]:
import pandas as pd
df=pd.read_csv('/content/resume_job_fit_combined.csv')
df

,resume_text,job_description_text,label
0,SummaryHighly motivated Sales Associate with e...,Net2Source Inc. is an award-winning total work...,No Fit
1,Professional SummaryCurrently working with Cat...,At Salas OBrien we tell our clients that were ...,No Fit
2,SummaryI started my construction career in Jun...,Schweitzer Engineering Laboratories (SEL) Infr...,No Fit
3,SummaryCertified Electrical Foremanwith thirte...,"Mizick Miller & Company, Inc. is looking for a...",No Fit
4,SummaryWith extensive experience in business/r...,Life at Capgemini\nCapgemini supports all aspe...,No Fit
...,...,...,...
7995,SummaryHighly motivated Sales Associate with e...,"Position Type: Full-Time, W2 Direct Hire. (Mus...",Good Fit
7996,SummaryWireless communications engineer with e...,"Location: Tampa, FL\nExp: 7-10 Yrs\nSPOC: Tush...",Good Fit
7997,Professional ProfileCapable International Tax ...,"Backed by a leading growth equity firm, an LA ...",Good Fit
7998,SummaryData Engineeringwith experience in Desi...,Allergan Data Labs is on a mission to transfor...,Good Fit


**Balance checking**

In [ ]:
df['label'].value_counts()

,count
label,
No Fit,4000
Potential Fit,2000
Good Fit,2000


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
import string

**Text Preprocessing for Resume-Job Matching using NLP**

In [ ]:
# Download the list of English stopwords from NLTK
nltk.download('stopwords')

# Import the stopwords list and create a set for faster lookup
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# Define a function to clean text data
def clean_text(text):
    # Replace newlines with spaces
    text = re.sub(r'\n', ' ', text)
    # Remove all punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Convert all characters to lowercase
    text = text.lower()
    # Tokenize (split) text into individual words
    tokens = text.split()
    # Remove stopwords from the token list
    tokens = [word for word in tokens if word not in stop_words]
    # Join the cleaned tokens back into a single string
    return ' '.join(tokens)

# Apply the cleaning function to the 'resume_text' column
df['resume_clean'] = df['resume_text'].apply(clean_text)

# Apply the cleaning function to the 'job_description_text' column
df['jd_clean'] = df['job_description_text'].apply(clean_text)

# Combine the cleaned resume and job description text into a single column
df['combined'] = df['resume_clean'] + " " + df['jd_clean']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.head()

,resume_text,job_description_text,label,resume_clean,jd_clean,combined
0,SummaryHighly motivated Sales Associate with e...,Net2Source Inc. is an award-winning total work...,No Fit,summaryhighly motivated sales associate extens...,net2source inc awardwinning total workforce so...,summaryhighly motivated sales associate extens...
1,Professional SummaryCurrently working with Cat...,At Salas OBrien we tell our clients that were ...,No Fit,professional summarycurrently working caterpil...,salas obrien tell clients engineered impact pa...,professional summarycurrently working caterpil...
2,SummaryI started my construction career in Jun...,Schweitzer Engineering Laboratories (SEL) Infr...,No Fit,summaryi started construction career june 2017...,schweitzer engineering laboratories sel infras...,summaryi started construction career june 2017...
3,SummaryCertified Electrical Foremanwith thirte...,"Mizick Miller & Company, Inc. is looking for a...",No Fit,summarycertified electrical foremanwith thirte...,mizick miller company inc looking dynamic indi...,summarycertified electrical foremanwith thirte...
4,SummaryWith extensive experience in business/r...,Life at Capgemini\nCapgemini supports all aspe...,No Fit,summarywith extensive experience businessrequi...,life capgemini capgemini supports aspects well...,summarywith extensive experience businessrequi...


**Data Upsampling**

In [ ]:
from sklearn.utils import resample

df_good = df[df.label == "Good Fit"]
df_potential = df[df.label == "Potential Fit"]
df_no = df[df.label == "No Fit"]

df_good_up = resample(df_good, replace=True, n_samples=4000, random_state=42)
df_potential_up = resample(df_potential, replace=True, n_samples=4000, random_state=42)

df_balanced = pd.concat([df_no, df_good_up, df_potential_up])
df_balanced['label'].value_counts()

,count
label,
No Fit,4000
Good Fit,4000
Potential Fit,4000


**TF-IDF Vectorization for Text Data**

In [ ]:
# Convert combined resume and job description text into numerical feature vectors
tfidf = TfidfVectorizer(max_features=5000)  # Use top 5000 most informative words based on TF-IDF scores

# Fit the TF-IDF model on the text data and transform it into a sparse matrix
X = tfidf.fit_transform(df_balanced['combined'])  # Feature matrix (input)

# Extract target labels for classification
y = df_balanced['label']  # Target variable (output)

**Data Splitting for training and testing**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

**Model training**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

**Model traning with 50 trees**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model50 = RandomForestClassifier(n_estimators=50, random_state=42)
model50.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, random_state=42)

**Model Training with 150 trees**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model150 = RandomForestClassifier(n_estimators=150, random_state=42)
model150.fit(X_train, y_train)

RandomForestClassifier(n_estimators=150, random_state=42)

**Model Training with 200 trees**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model200 = RandomForestClassifier(n_estimators=200, random_state=42)
model200.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

**Visualize the tree**

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
plt.figure(figsize=(100, 80))
plot_tree(
    model.estimators_[0],
    feature_names=tfidf.get_feature_names_out(),
    class_names=model.classes_,
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title("Random Forest - Zoomed Decision Tree", fontsize=20)
plt.show()

**predicting values**

In [ ]:
y_pred = model.predict(X_test)
import pandas as pd

comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(comparison_df.head(20))  # show top 20 rows

             Actual      Predicted
5196       Good Fit       Good Fit
1629         No Fit       Good Fit
5259       Good Fit       Good Fit
6251         No Fit         No Fit
7852       Good Fit       Good Fit
4656  Potential Fit  Potential Fit
7764       Good Fit       Good Fit
6038       Good Fit       Good Fit
3290  Potential Fit  Potential Fit
1959         No Fit         No Fit
5928       Good Fit       Good Fit
7476  Potential Fit         No Fit
4113  Potential Fit  Potential Fit
6228       Good Fit       Good Fit
1107         No Fit         No Fit
7338  Potential Fit  Potential Fit
3828  Potential Fit  Potential Fit
5742       Good Fit       Good Fit
4213  Potential Fit  Potential Fit
6324         No Fit         No Fit


**Metrics measurering**

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
# Accuracy
acc = accuracy_score(y_test, y_pred)

f1 = f1_score(y_test, y_pred, average='macro')

print("Accuracy Score:", acc)
print("F1 Score (Macro):", f1)
print("Precision Score:", precision_score(y_test, y_pred, average='macro'))
print("Recall Score:", recall_score(y_test, y_pred, average='macro'))

Accuracy Score: 0.8425
F1 Score (Macro): 0.839337152378393
Precision Score: 0.8421312326936151
Recall Score: 0.8424999999999999


**Saving model and vectorizer**

In [ ]:
import joblib

# Save the vectorizer
joblib.dump(tfidf, 'tfidf_vectorizer.joblib')

# Save the trained model
joblib.dump(model, 'resume_classifier_model.joblib')

['resume_classifier_model.joblib']

**Install libraries for UI**

In [ ]:
!pip install gradio docx2txt pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.6 MB/s eta 0:00:00


**UI**

In [ ]:
import gradio as gr
import joblib
import pdfplumber
import docx2txt
import os
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords (first time only)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load model and vectorizer
model = joblib.load('resume_classifier_model.joblib')
tfidf = joblib.load('tfidf_vectorizer.joblib')

# Text Cleaning Function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# Extract text from PDF
def extract_text_from_pdf(file):
    with pdfplumber.open(file.name) as pdf:
        text = ''
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

# Extract text from DOCX
def extract_text_from_docx(file):
    return docx2txt.process(file.name)

# Predict Fit for Multiple Resumes
def predict_fit(resume_files, job_description):
    all_predictions = []  # To store predictions for each resume file

    for resume_file in resume_files:  # Iterate over each uploaded resume file
        ext = os.path.splitext(resume_file.name)[1].lower()

        # Extract text from different file types
        if ext == ".pdf":
            resume_text = extract_text_from_pdf(resume_file)
        elif ext == ".docx":
            resume_text = extract_text_from_docx(resume_file)
        else:
            all_predictions.append(f"{os.path.basename(resume_file.name)}: ❌ Unsupported file format. Please upload a .pdf or .docx file.")
            continue

        # Clean the texts
        cleaned_resume = clean_text(resume_text)
        cleaned_jd = clean_text(job_description)

        # Combine resume text with job description
        combined = cleaned_resume + " " + cleaned_jd
        X_input = tfidf.transform([combined])

        # Make prediction
        prediction = model.predict(X_input)[0]
        all_predictions.append(f"{os.path.basename(resume_file.name)}: {prediction}")

    return "\n".join(all_predictions)  # Return the results for all uploaded resumes

# Gradio Interface for Batch Processing
interface = gr.Interface(
    fn=predict_fit,
    inputs=[
        gr.Files(label="Upload Resumes (.pdf or .docx)"),  # Allow multiple file uploads
        gr.Textbox(lines=6, label="Paste Job Description")
    ],
    outputs="text",
    title="AI Resume Analyzer",
    description="Upload multiple resumes and paste a job description. The AI will clean the text and predict: Good Fit, Potential Fit, or No Fit for each resume."
)

interface.launch(debug=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d65e9ddf586308f96f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
